In [1]:
from keccak import *
from binascii import hexlify, unhexlify

import unittest
import os
from hashlib import sha3_256, sha3_512, shake_128, shake_256
from kyber import Kyber512, Kyber768, Kyber1024
from aes256_ctr_drbg import AES256_CTR_DRBG

def parse_kat_data(data):
    parsed_data = {}
    count_blocks = data.split('\n\n')
    for block in count_blocks[1:-1]:
        block_data = block.split('\n')
        count, seed, pk, sk, ct, ss = [line.split(" = ")[-1] for line in block_data]
        parsed_data[count] = {
            "seed": bytes.fromhex(seed),
            "pk": bytes.fromhex(pk),
            "sk": bytes.fromhex(sk),
            "ct": bytes.fromhex(ct),
            "ss": bytes.fromhex(ss),   
        }
    return parsed_data
    


In [2]:
with open("assets/PQCkemKAT_1632.rsp") as f:
    kat_data = f.read()
    parsed_data = parse_kat_data(kat_data)

data = parsed_data['0']
seed, pk, sk, ct, ss = data.values()
        
# Seed DRBG with KAT seed
Kyber512.set_drbg_seed(seed)
# Assert keygen matches
_pk, _sk = Kyber512.keygen()

# Assert encapsulation matches
_ct, _ss = Kyber512.enc(_pk)

#    # Assert decapsulation matches
#    __ss = Kyber512.dec(ct, sk)

In [3]:
seed.hex()

'061550234d158c5ec95595fe04ef7a25767f2e24cc2bc479d09d86dc9abcfde7056a8c266f9ef97ed08541dbd2e1ffa1'

In [4]:
pk == _pk

True

In [5]:
(pk == _pk) == (sk == _sk) == (ct == _ct) == (ss == _ss)

True

In [6]:
random_bytes = os.urandom

In [7]:
a = random_bytes(32)

In [8]:
len(Kyber512._xof_a)

1

In [9]:
len(Kyber512._xof_input_bytes)

34

In [10]:
Kyber512._xof_a.hex()

'01'

In [11]:
shake_256(Kyber512._xof_input_bytes).digest(Kyber512._xof_length).hex()

'f3210df1ba3dc9392d9ee5fcbe267736dc79ac3f9945884239913ffc1bffd0101c5ff800daa6e7068eee8ae125b99784ec3e1bd11b712cacc4fbf409a08d054c000706b5d07aa7827710fb4d5a923434bb60c2c1335f63fed6c5a90db4f7c5eada6cf30ffcc09ac717ec1ed72cea5f299bd43b4fea07d911c5d09f0505bc10e2ef4e65cc38f28c7010495523d9dd02aace467401858fd62926f0b36a3ec7f2b565cea026dedba950d3dca7cd5c038314084e3174450fc80c5508b49254215f052a48a1794b58b84dfb4733123a3c93ee46456e29d241abf7b634d855b7eb7c90f1c281988a4c677e503b4c03d1ba3f00e4f58c3e8795f27f394459cb9988f7db6120be7b354475287c2647117a72acb350132afab0192e662b345995638df29a2ec390a2cd77909eaf15077b5d5cabf6467f17f166d8cff9161a11573c369b2e7e8ab74b8d2a6b03856f0389bd890c1ad3015ef6a842e90eac1db29f8a3919ebc58e6741071dacc920d74eb36a22f513f2f04841dbe0c7931de93c0dad29e9219a82a18152b25c326b504decd7017edbbb26ad9a1466563b5a127077f695468bb7a48e11f524ec62b8b81532c3ebc3a7d5311a5d166eb4ece66bf7e7d9a1f420d37c1220d654a9133e7b8d7517e2ffdad5e4445c3fc2e729223330aa6a8904abdca58cf4c8c0d78eb63fc6fda5429f7f092b3a6

In [12]:
SHAKE_256(Kyber512._xof_input_bytes).squeeze(Kyber512._xof_length).hex()

'f3210df1ba3dc9392d9ee5fcbe267736dc79ac3f9945884239913ffc1bffd0101c5ff800daa6e7068eee8ae125b99784ec3e1bd11b712cacc4fbf409a08d054c000706b5d07aa7827710fb4d5a923434bb60c2c1335f63fed6c5a90db4f7c5eada6cf30ffcc09ac717ec1ed72cea5f299bd43b4fea07d911c5d09f0505bc10e2ef4e65cc38f28c7010495523d9dd02aace467401858fd62926f0b36a3ec7f2b565cea026dedba950d3dca7cd5c038314084e3174450fc80c5508b49254215f052a48a1794b58b84dfb4733123a3c93ee46456e29d241abf7b634d855b7eb7c90f1c281988a4c677e503b4c03d1ba3f00e4f58c3e8795f27f394459cb9988f7db6120be7b354475287c2647117a72acb350132afab0192e662b345995638df29a2ec390a2cd77909eaf15077b5d5cabf6467f17f166d8cff9161a11573c369b2e7e8ab74b8d2a6b03856f0389bd890c1ad3015ef6a842e90eac1db29f8a3919ebc58e6741071dacc920d74eb36a22f513f2f04841dbe0c7931de93c0dad29e9219a82a18152b25c326b504decd7017edbbb26ad9a1466563b5a127077f695468bb7a48e11f524ec62b8b81532c3ebc3a7d5311a5d166eb4ece66bf7e7d9a1f420d37c1220d654a9133e7b8d7517e2ffdad5e4445c3fc2e729223330aa6a8904abdca58cf4c8c0d78eb63fc6fda5429f7f092b3a6

In [13]:
a

b'\x8a\xa9\x1f7}7H\x01\xf5\xcfa\xf5\xc9\xd0\xf0LG\x11\xad\x13Z\x92\xfc\x9ab\xd6\xb57\xfe3\xc6N'

In [14]:
pt = unhexlify(
    "023D91AC532601C7CA3942D62827566D9268BB4276FCAA1AE927693A6961652676DBA09219A01B3D5ADFA12547A946E78F3C5C62DD880B02D2EEEB4B96636529C6B01120B23EFC49CCFB36B8497CD19767B53710A636683BC5E0E5C9534CFC004691E87D1BEE39B86B953572927BD668620EAB87836D9F3F8F28ACE41150776C0BC6657178EBF297FE1F7214EDD9F215FFB491B681B06AC2032D35E6FDF832A8B06056DA70D77F1E9B4D26AE712D8523C86F79250718405F91B0A87C725F2D3F52088965F887D8CF87206DFDE422386E58EDDA34DDE2783B3049B86917B4628027A05D4D1F429D2B49C4B1C898DDDCB82F343E145596DE11A54182F39F4718ECAE8F506BD9739F5CD5D5686D7FEFC834514CD1B2C91C33B381B45E2E5335D7A8720A8F17AFC8C2CB2BD88B14AA2DCA099B00AA575D0A0CCF099CDEC4870FB710D2680E60C48BFC291FF0CEF2EEBF9B36902E9FBA8C889BF6B4B9F5CE53A19B0D9399CD19D61BD08C0C2EC25E099959848E6A550CA7137B63F43138D7B651"
)
expect = "230620d710cf3ab835059e1aa170735db17cae74b345765ff02e8d89"
h = Keccak224(pt).hexdigest()
assert h == expect

In [15]:
h

'230620d710cf3ab835059e1aa170735db17cae74b345765ff02e8d89'